**INSTALL DEPENDENCIES AND UPLOAD NECESSARY DATASETS**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
top_60_df=pd.read_csv('final_hopefully.csv')
top_60_df.head()

,Unnamed: 0.1,Unnamed: 0,YEAR,TRACTFIPS,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,TERRITORY,...,ACS_PCT_AGE_30_44,ACS_PCT_AGE_45_64,ACS_PCT_AGE_ABOVE65,ACS_PCT_MALE,ACS_PCT_VET,ACS_PER_CAPITA_INC,ACS_PCT_LT_HS,ACS_MEDIAN_RENT,ACS_PCT_HH_1PERS,ACS_PCT_UNINSURED
0,0,0,2020,1001020100,1001,1,Alabama,Autauga County,South,0,...,21.54,28.23,15.20,50.39,11.73,30447.0,14.29,820.0,18.61,9.63
1,1,1,2020,1001020200,1001,1,Alabama,Autauga County,South,0,...,20.83,22.99,16.16,57.48,10.35,22772.0,10.59,846.0,33.33,5.94
2,2,2,2020,1001020300,1001,1,Alabama,Autauga County,South,0,...,22.69,25.20,12.56,49.78,13.70,25291.0,12.80,937.0,22.65,3.46
3,3,3,2020,1001020400,1001,1,Alabama,Autauga County,South,0,...,17.24,25.35,27.38,50.69,15.24,47056.0,6.23,917.0,25.18,4.79
4,4,4,2020,1001020501,1001,1,Alabama,Autauga County,South,0,...,14.56,27.06,12.56,50.84,18.46,30857.0,8.80,1505.0,25.09,3.19


In [ ]:
tract_top_60=top_60_df['TRACTFIPS']
tract_top_60 = pd.DataFrame({'cencus_tract': tract_top_60})

In [ ]:
cencus_zip_df=pd.read_csv('censusdatawithzip.csv')
cencus_zip_df.head()

,Unnamed: 0,YEAR,TRACTFIPS,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,TERRITORY,ACS_TOT_POP_WT,...,HIFLD_DIST_UC_TRACT,POS_DIST_ED_TRACT,POS_DIST_MEDSURG_ICU_TRACT,POS_DIST_TRAUMA_TRACT,POS_DIST_PED_ICU_TRACT,POS_DIST_OBSTETRICS_TRACT,POS_DIST_CLINIC_TRACT,POS_DIST_ALC_TRACT,CEN_AIAN_NH_IND,ZIP
0,0,2020,1001020100,1001,1,Alabama,Autauga County,South,0,1941.0,...,3.52,2.25,2.25,12.72,55.04,12.72,10.30,13.11,0,36067.0
1,1,2020,1001020200,1001,1,Alabama,Autauga County,South,0,1757.0,...,2.64,1.36,1.36,11.89,55.65,11.89,10.99,12.24,0,36067.0
2,2,2020,1001020300,1001,1,Alabama,Autauga County,South,0,3694.0,...,1.99,0.85,0.85,11.48,55.75,11.48,10.98,11.71,0,36067.0
3,3,2020,1001020400,1001,1,Alabama,Autauga County,South,0,3539.0,...,0.97,0.55,0.55,10.55,56.49,10.55,9.95,10.70,0,36066.0
4,4,2020,1001020501,1001,1,Alabama,Autauga County,South,0,4306.0,...,0.69,1.79,1.79,8.95,57.99,8.95,8.55,9.13,0,36066.0


In [ ]:
cencus_tract_all=cencus_zip_df['TRACTFIPS']
cencus_tract_all.shape

zip_all=cencus_zip_df['ZIP']
zip_all.shape

# Creating DataFrame using the Series
zip_tract_df = pd.DataFrame({'cencus_tract': cencus_tract_all, 'zip': zip_all})
zip_tract_df.head()

merged_tract_zip = pd.merge(left=tract_top_60, right=zip_tract_df, on='cencus_tract', how='inner')
merged_tract_zip.shape

merged_tract_clean=merged_tract_zip.dropna()

top_60_df_new=top_60_df.drop(columns=['Unnamed: 0.1','TRACTFIPS'])

top_60_df_new=pd.concat([merged_tract_zip[['cencus_tract','zip']],top_60_df_new],axis=1)

top_60_df_new.loc[top_60_df_new.isnull().any(axis=1)]

top_60_df_new=top_60_df_new.dropna()

zip_new=top_60_df_new['zip'].to_list()

top_60_df_new.insert(0,'zipcode',zip_new)
top_60_df_zip=top_60_df_new.drop(columns='zip')

top_60_zip=top_60_df_zip.drop(columns=['TERRITORY','YEAR']) #<-------------------------

cencus_zip_df.rename(columns={'TRACTFIPS': 'cencus_tract'}, inplace=True) #match the column name

cols=cencus_zip_df.keys()
cols=cols[:9]

cencus_zip_new=cencus_zip_df.drop(columns=cols)

cen_zip=top_60_zip['cencus_tract'] #mache the rows based on the cencus_tract


ord_cencus_all = pd.merge(left=cen_zip, right=cencus_zip_df, on='cencus_tract', how='inner')

ord_cencus_all.fillna(0,inplace=True)#<-------------------------------------

col=top_60_zip.keys()
col_bad=col[:7]
top_60_feature_only=top_60_zip.drop(columns=col_bad)

# usable csv files---> top_60_zip,ord_cencus_all

In [ ]:
top_60_zip.head()

,zipcode,cencus_tract,Unnamed: 0,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,ACS_PCT_HH_SMARTPHONE_ONLY,ACS_PCT_AGE_15_17,...,ACS_PCT_AGE_30_44,ACS_PCT_AGE_45_64,ACS_PCT_AGE_ABOVE65,ACS_PCT_MALE,ACS_PCT_VET,ACS_PER_CAPITA_INC,ACS_PCT_LT_HS,ACS_MEDIAN_RENT,ACS_PCT_HH_1PERS,ACS_PCT_UNINSURED
0,36067.0,1001020100,0,1001,1,Alabama,Autauga County,South,5.92,2.73,...,21.54,28.23,15.20,50.39,11.73,30447.0,14.29,820.0,18.61,9.63
1,36067.0,1001020200,1,1001,1,Alabama,Autauga County,South,12.91,5.29,...,20.83,22.99,16.16,57.48,10.35,22772.0,10.59,846.0,33.33,5.94
2,36067.0,1001020300,2,1001,1,Alabama,Autauga County,South,13.69,4.66,...,22.69,25.20,12.56,49.78,13.70,25291.0,12.80,937.0,22.65,3.46
3,36066.0,1001020400,3,1001,1,Alabama,Autauga County,South,5.13,2.32,...,17.24,25.35,27.38,50.69,15.24,47056.0,6.23,917.0,25.18,4.79
4,36066.0,1001020501,4,1001,1,Alabama,Autauga County,South,0.00,6.57,...,14.56,27.06,12.56,50.84,18.46,30857.0,8.80,1505.0,25.09,3.19


In [ ]:

first_column = top_60_zip.pop('Unnamed: 0')
top_60_zip.insert(0, 'Unnamed: 0', first_column)
top_60_zip.head()

,Unnamed: 0,zipcode,cencus_tract,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,ACS_PCT_HH_SMARTPHONE_ONLY,ACS_PCT_AGE_15_17,...,ACS_PCT_AGE_30_44,ACS_PCT_AGE_45_64,ACS_PCT_AGE_ABOVE65,ACS_PCT_MALE,ACS_PCT_VET,ACS_PER_CAPITA_INC,ACS_PCT_LT_HS,ACS_MEDIAN_RENT,ACS_PCT_HH_1PERS,ACS_PCT_UNINSURED
0,0,36067.0,1001020100,1001,1,Alabama,Autauga County,South,5.92,2.73,...,21.54,28.23,15.20,50.39,11.73,30447.0,14.29,820.0,18.61,9.63
1,1,36067.0,1001020200,1001,1,Alabama,Autauga County,South,12.91,5.29,...,20.83,22.99,16.16,57.48,10.35,22772.0,10.59,846.0,33.33,5.94
2,2,36067.0,1001020300,1001,1,Alabama,Autauga County,South,13.69,4.66,...,22.69,25.20,12.56,49.78,13.70,25291.0,12.80,937.0,22.65,3.46
3,3,36066.0,1001020400,1001,1,Alabama,Autauga County,South,5.13,2.32,...,17.24,25.35,27.38,50.69,15.24,47056.0,6.23,917.0,25.18,4.79
4,4,36066.0,1001020501,1001,1,Alabama,Autauga County,South,0.00,6.57,...,14.56,27.06,12.56,50.84,18.46,30857.0,8.80,1505.0,25.09,3.19


In [ ]:
ord_cencus_all.head()

,cencus_tract,Unnamed: 0,YEAR,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,TERRITORY,ACS_TOT_POP_WT,...,HIFLD_DIST_UC_TRACT,POS_DIST_ED_TRACT,POS_DIST_MEDSURG_ICU_TRACT,POS_DIST_TRAUMA_TRACT,POS_DIST_PED_ICU_TRACT,POS_DIST_OBSTETRICS_TRACT,POS_DIST_CLINIC_TRACT,POS_DIST_ALC_TRACT,CEN_AIAN_NH_IND,ZIP
0,1001020100,0,2020,1001,1,Alabama,Autauga County,South,0,1941.0,...,3.52,2.25,2.25,12.72,55.04,12.72,10.30,13.11,0,36067.0
1,1001020200,1,2020,1001,1,Alabama,Autauga County,South,0,1757.0,...,2.64,1.36,1.36,11.89,55.65,11.89,10.99,12.24,0,36067.0
2,1001020300,2,2020,1001,1,Alabama,Autauga County,South,0,3694.0,...,1.99,0.85,0.85,11.48,55.75,11.48,10.98,11.71,0,36067.0
3,1001020400,3,2020,1001,1,Alabama,Autauga County,South,0,3539.0,...,0.97,0.55,0.55,10.55,56.49,10.55,9.95,10.70,0,36066.0
4,1001020501,4,2020,1001,1,Alabama,Autauga County,South,0,4306.0,...,0.69,1.79,1.79,8.95,57.99,8.95,8.55,9.13,0,36066.0


In [ ]:
first_column = ord_cencus_all.pop('Unnamed: 0')
ord_cencus_all.insert(0, 'Unnamed: 0', first_column)
ord_cencus_all.head()

,Unnamed: 0,cencus_tract,YEAR,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,TERRITORY,ACS_TOT_POP_WT,...,HIFLD_DIST_UC_TRACT,POS_DIST_ED_TRACT,POS_DIST_MEDSURG_ICU_TRACT,POS_DIST_TRAUMA_TRACT,POS_DIST_PED_ICU_TRACT,POS_DIST_OBSTETRICS_TRACT,POS_DIST_CLINIC_TRACT,POS_DIST_ALC_TRACT,CEN_AIAN_NH_IND,ZIP
0,0,1001020100,2020,1001,1,Alabama,Autauga County,South,0,1941.0,...,3.52,2.25,2.25,12.72,55.04,12.72,10.30,13.11,0,36067.0
1,1,1001020200,2020,1001,1,Alabama,Autauga County,South,0,1757.0,...,2.64,1.36,1.36,11.89,55.65,11.89,10.99,12.24,0,36067.0
2,2,1001020300,2020,1001,1,Alabama,Autauga County,South,0,3694.0,...,1.99,0.85,0.85,11.48,55.75,11.48,10.98,11.71,0,36067.0
3,3,1001020400,2020,1001,1,Alabama,Autauga County,South,0,3539.0,...,0.97,0.55,0.55,10.55,56.49,10.55,9.95,10.70,0,36066.0
4,4,1001020501,2020,1001,1,Alabama,Autauga County,South,0,4306.0,...,0.69,1.79,1.79,8.95,57.99,8.95,8.55,9.13,0,36066.0


In [ ]:
ip_dict2 = {'zipcode':123,'Address':None,'Gender':None,'age':None,'Race':None,'Income':None,'Education':None,'Veteran Status':None}

* flag_1=address present or not
* flag_2=gender present or not
* flag_3=age present or not
* flag_4= race or not
* flag_5=income present or not
* flag_6=edu present or not
* flag_7=veteran status present or not

**Age Data Imputation Function**

This function, `impute_age`, is designed to impute age-related data based on user inputs and demographic information from a census dataset. It adjusts for age-specific categories and gender, updating the output with calculated values to reflect demographic proportions accurately. This process includes:

- Assigning age-specific percentage columns based on input age and gender.
- Calculating values for predefined features across different census data points.
- Clearing and updating output data structures with new imputed values for further analysis.

This method is particularly useful in datasets where age and gender demographics play a crucial role in the analysis, ensuring the data accurately represents the studied population.


In [ ]:
def impute_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices):
  # age ranges-> 15-17,0-17,30-44,45-64,>65
  pop_total=0
  if(input['age']<17 and input['age']>15):


#add the age specific percentage column+
    age_specific_col=[]
    for index in indices:
        age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_15_17'])

    cencus_zip['ACS_PCT_AGE_15_17']=age_specific_col

    if(input['Gender']==None):

      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_15_17']

          val+=df_60.loc[i,feat]*age_pop
          pop_deno+=age_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='M'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_15_17']*cencus_zip.loc[i,'male_percent']

          val+=df_60.loc[i,feat]*age_male_pop
          pop_deno+=age_male_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='F'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_15_17']*cencus_zip.loc[i,'female_percent']

          val+=df_60.loc[i,feat]*age_female_pop
          pop_deno+=age_female_pop

        val/=pop_deno
        output[feat].append(val)

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(100)
    output['ACS_PCT_AGE_0_17'].append(0)
    output['ACS_PCT_AGE_30_44'].append(0)
    output['ACS_PCT_AGE_45_64'].append(0)
    output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<17):


    age_specific_col=[]
    for index in indices:
        age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_0_17'])

    cencus_zip['ACS_PCT_AGE_0_17']=age_specific_col

    if(input['Gender']==None):

      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_0_17']

          val+=df_60.loc[i,feat]*age_pop
          pop_deno+=age_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='M'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_0_17']*cencus_zip.loc[i,'male_percent']

          val+=df_60.loc[i,feat]*age_male_pop
          pop_deno+=age_male_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='F'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_0_17']*cencus_zip.loc[i,'female_percent']

          val+=df_60.loc[i,feat]*age_female_pop
          pop_deno+=age_female_pop

        val/=pop_deno
        output[feat].append(val)

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(0)
    output['ACS_PCT_AGE_0_17'].append(100)
    output['ACS_PCT_AGE_30_44'].append(0)
    output['ACS_PCT_AGE_45_64'].append(0)
    output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<44 and input['age']>30):


    if(input['Gender']==None):

      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_30_44']

          val+=df_60.loc[i,feat]*age_pop
          pop_deno+=age_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='M'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_30_44']*cencus_zip.loc[i,'male_percent']

          val+=df_60.loc[i,feat]*age_male_pop
          pop_deno+=age_male_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='F'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_30_44']*cencus_zip.loc[i,'female_percent']

          val+=df_60.loc[i,feat]*age_female_pop
          pop_deno+=age_female_pop

        val/=pop_deno
        output[feat].append(val)

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(0)
    output['ACS_PCT_AGE_0_17'].append(0)
    output['ACS_PCT_AGE_30_44'].append(100)
    output['ACS_PCT_AGE_45_64'].append(0)
    output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<=64 and input['age']>45):


    if(input['Gender']==None):

      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_45_64']

          val+=df_60.loc[i,feat]*age_pop
          pop_deno+=age_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='M'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_45_64']*cencus_zip.loc[i,'male_percent']

          val+=df_60.loc[i,feat]*age_male_pop
          pop_deno+=age_male_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='F'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_45_64']*cencus_zip.loc[i,'female_percent']

          val+=df_60.loc[i,feat]*age_female_pop
          pop_deno+=age_female_pop

        val/=pop_deno
        output[feat].append(val)

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(0)
    output['ACS_PCT_AGE_0_17'].append(0)
    output['ACS_PCT_AGE_30_44'].append(0)
    output['ACS_PCT_AGE_45_64'].append(100)
    output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']>=65):


    if(input['Gender']==None):

      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_ABOVE65']

          val+=df_60.loc[i,feat]*age_pop
          pop_deno+=age_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='M'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_ABOVE65']*cencus_zip.loc[i,'male_percent']

          val+=df_60.loc[i,feat]*age_male_pop
          pop_deno+=age_male_pop

        val/=pop_deno
        output[feat].append(val)

    elif(input['Gender']=='F'):
      #use the elements of cencus_zip['male_percent']
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          age_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AGE_ABOVE65']*cencus_zip.loc[i,'female_percent']

          val+=df_60.loc[i,feat]*age_female_pop
          pop_deno+=age_female_pop

        val/=pop_deno
        output[feat].append(val)

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(0)
    output['ACS_PCT_AGE_0_17'].append(0)
    output['ACS_PCT_AGE_30_44'].append(0)
    output['ACS_PCT_AGE_45_64'].append(0)
    output['ACS_PCT_AGE_ABOVE65'].append(100)
  else:

    # if('ACS_PCT_AGE_15_17' in list(output.keys())):
    #   print('yes')
    print('no age group matcehs')
    print(output)
    output,cencus_zip=impute_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)

    print(output)
    print(type(output['ACS_PCT_AGE_15_17']))

    output['ACS_PCT_AGE_15_17'].clear()
    output['ACS_PCT_AGE_0_17'].clear()
    output['ACS_PCT_AGE_30_44'].clear()
    output['ACS_PCT_AGE_45_64'].clear()
    output['ACS_PCT_AGE_ABOVE65'].clear()

    output['ACS_PCT_AGE_15_17'].append(0)
    output['ACS_PCT_AGE_0_17'].append(0)
    output['ACS_PCT_AGE_30_44'].append(0)
    output['ACS_PCT_AGE_45_64'].append(0)
    output['ACS_PCT_AGE_ABOVE65'].append(0)
    #set the other features same as decided by gender only.

  return output,cencus_zip #collect the new cencus_zip for future reference

**Gender Data Integration in Census Data**

The function `make_cencus_zip_gend` integrates gender-specific data into a census dataset, enhancing its demographic granularity. It operates by:

- Iterating over specified indices to collect male and female population percentages from the ordered census dataset (`ord_cencus_all`).
- Assigning these percentages to a new or existing `census_zip` DataFrame, creating or updating `male_percent` and `female_percent` columns.

This enhancement allows for gender-specific analysis within the dataset, supporting more nuanced insights into demographic distributions and trends. The inclusion of comments with question marks indicates areas where clarification or decisions regarding data treatment may be necessary, especially concerning population changes.


In [ ]:
def make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices):

  #---------------------???????????????Ask Regarding the change in population??????????????????-----------------------------
  male_percentage_per_cencus=[]
  for index in indices:
    male_percentage_per_cencus.append(ord_cencus_all.loc[index,'ACS_PCT_MALE'])

  cencus_zip['male_percent']=male_percentage_per_cencus #the cencus_zip contains male percentage column

  #---------------------???????????????Ask Regarding the change in population??????????????????-----------------------------
  female_percentage_per_cencus=[]
  for index in indices:
    female_percentage_per_cencus.append(ord_cencus_all.loc[index,'ACS_PCT_FEMALE'])

  cencus_zip['female_percent']=female_percentage_per_cencus

  return cencus_zip

In [ ]:
def impute_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices):
  pop_total=0

  if(input['Gender']=='M'):
    print('Launda')

    #---------------------???????????????Ask Regarding the change in population??????????????????-----------------------------
    male_percentage_per_cencus=[]
    for index in indices:
      male_percentage_per_cencus.append(ord_cencus_all.loc[index,'ACS_PCT_MALE'])

    cencus_zip['male_percent']=male_percentage_per_cencus #the cencus_zip contains male percentage column

    #as only male column present in the final dataframe
    for feat in top_60_feature_only.keys(): #for all features
      val=0

      for i in range(cencus_zip.shape[0]):#for all the cencus
        male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'male_percent']
        val+=df_60.loc[i,feat]*male_pop
        pop_total+=male_pop
        #---------------------????????????????change the total population number based on the percentage of population male?????????-----------------

      val/=pop_total
      output[feat].append(val) #as it is a 1 element list

    #out_df = pd.DataFrame.from_dict(output)
    output['ACS_PCT_MALE']=100 #all male


  elif(input['Gender']=='F'):
        print('Laundi')

        #---------------------???????????????Ask Regarding the change in population??????????????????-----------------------------
        female_percentage_per_cencus=[]
        for index in indices:
          female_percentage_per_cencus.append(ord_cencus_all.loc[index,'ACS_PCT_FEMALE'])

        cencus_zip['female_percent']=female_percentage_per_cencus
        print('cencus_zip------')
        print(cencus_zip)
        print('cencus_zip------')
        #as only male column present in the final dataframe
        for feat in list(top_60_feature_only.keys()): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'female_percent']

            #print('female pop: ',female_pop)
            #print('feat: ',feat)
            #print('feature value of the row: ',cencus_zip.loc[i,feat])

            val+=cencus_zip.loc[i,feat]*female_pop
            pop_total+=female_pop

          val/=pop_total
          #print('value of ',feat,' column is ',val)
          output[feat].append(val) #as it is a 1 element list
          #print(output)

        output['ACS_PCT_MALE']=0 #all female
  print(type(output))
  return output,cencus_zip #to access the column male percent/female percent

In [ ]:
def check_flag(input):
  keys=list(input.keys())
  zip_less_keys=keys[1:]
  flag_add=False
  flag_gen=False
  flag_age=False
  flag_race=False
  flag_income=False
  flag_edu=False
  flag_vet=False
  flag_dict={'flag_add':None,'flag_gen':None,'flag_age':None,'flag_race':None,'flag_income':None,'flag_edu':None,'flag_vet':None}

  for key in zip_less_keys:
        if(key=='Address' and input[key]!=None):
          flag_add=True
          flag_dict['flag_add']=True

        if(key=='Gender' and input[key]!=None):
          flag_gen=True
          flag_dict['flag_gen']=True

        if(key=='age' and input[key]!=None):
          flag_age=True
          flag_dict['flag_age']=True

        if(key=='Race' and input[key]!=None):
          flag_race=True
          flag_dict['flag_race']=True

        if(key=='Income' and input[key]!=None):
          flag_income=True
          flag_dict['flag_income']=True

        if(key=='Education' and input[key]!=None):
          flag_edu=True
          flag_dict['flag_edu']=True

        if(key=='Veteran Status' and input[key]!=None):
          flag_vet=True
          flag_dict['flag_vet']=True

  return flag_dict

In [ ]:
def find_cencus(input,df_60,ord_cencus_all):
  '''
  input: a dictionary of the features given by the user
  df_60: the final_hopefully dataset with all the cencus tracts and zipcodes corresponding to them present
  ord_cencus_all: the cencuswithzip dataset, containing all the cencus tracts and corresponding zipcodes and the feature names(population number also)
  '''
  col=df_60.keys()
  col_bad=col[:8]
  top_60_feature_only=df_60.drop(columns=col_bad) #refer to obtain the features

  #zipcode will be always given
  cencus_zip=df_60.loc[df_60['zipcode']==input['zipcode']].reset_index(drop=True)#obtain the cencus tracts from the df_60
  print('zipcode found')
  #print(cencus_zip)
  #return cencus_zip
  total_population=[]
  indices=[]
  for row in range(cencus_zip.shape[0]): #for all counties
    index=0
    #print('row:',row )
    #print('finding: ',cencus_zip.loc[row,'cencus_tract'])
    for loc in range(ord_cencus_all.shape[0]):
      #print('loc: ',loc )
      if(ord_cencus_all.loc[loc,'cencus_tract'] == cencus_zip.loc[row,'cencus_tract']):
        index=loc
        indices.append(index)
        #print('index got :',index)
        #print('found: ',ord_cencus_all.loc[loc,'cencus_tract'])
        break

    #index  = ord_cencus_all.index[ord_cencus_all['cencus_tract'] == cencus_zip.loc[row,'cencus_tract']].tolist()[0] #find the index of the all feature dataframe where the cencus tract matches


    total_population.append(ord_cencus_all.loc[index,'ACS_TOT_POP_US_ABOVE1']) #extract the values of total population from the oredered cencus_zip dataframe

  #print('total population in each cencus tract',total_population)

  cencus_zip['population']=total_population

  print('cencus_zip returned from find_cencus function-------------')
  print(cencus_zip)
  return cencus_zip,top_60_feature_only,total_population,indices

In [ ]:
def normal_impute(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output):

  print('only zipcode given')
  #do the wghted average
  population_tot=sum(total_population)
  print('total population: ',population_tot)
  #Later:- do weighted average only on the percentage features. Leave the population like numbers.
  #weighted average on all the features
  for feat in top_60_feature_only.keys(): #for all features
    val=0
    for i in range(cencus_zip.shape[0]):#for all the cencus
      val+=df_60.loc[i,feat]*cencus_zip.loc[i,'population']

    val/=population_tot
    output[feat].append(val)

    #print(output)
  return output

In [ ]:
type(ip_dict2)

dict

In [ ]:
def make_df(output,address=None,state=None,county=None):
  Address=0
  State=''
  County=''

  if(type(output) is dict):
    print('after imputation, the dict---------')
    print(output)
    print('the dict---------------')
    keys=list(output.keys())

    if(address!=None):
        Address=address
        print('address: ',Address)
    if(state!=None):
      State=state
      print('state: ',State)
    if(county!=None):
      County=county
      print('county: ',County)

    if(type(output[keys[0]])==list):
      out_df = pd.DataFrame.from_dict(output)

      out_df.insert(0,'TRACTFIPS',[Address])
      out_df.insert(0,'STATE',[state])
      out_df.insert(0,'COUNTY',[county])
    else:
      out_df = pd.DataFrame.from_dict(output, orient='index', columns=['Values'])
      out_df=out_df.T
      out_df = out_df.rename_axis(None, axis=0)
      out_df.rows = [0]

      out_df.insert(0,'TRACTFIPS',Address)
      out_df.insert(0,'STATE',state)
      out_df.insert(0,'COUNTY',county)

    return out_df
  else:
    return output

In [ ]:
def onehot_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices) :#give the cencus_zip_gend:
  if(input['age']<17 and input['age']>15):

      #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      output['ACS_PCT_AGE_15_17']=0
      output['ACS_PCT_AGE_0_17']=0
      output['ACS_PCT_AGE_30_44']=0
      output['ACS_PCT_AGE_45_64']=0
      output['ACS_PCT_AGE_ABOVE65']=0

      # output['ACS_PCT_AGE_15_17'].append(100)
      # output['ACS_PCT_AGE_0_17'].append(0)
      # output['ACS_PCT_AGE_30_44'].append(0)
      # output['ACS_PCT_AGE_45_64'].append(0)
      # output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<17):

      #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      output['ACS_PCT_AGE_15_17']=0
      output['ACS_PCT_AGE_0_17']=0
      output['ACS_PCT_AGE_30_44']=0
      output['ACS_PCT_AGE_45_64']=0
      output['ACS_PCT_AGE_ABOVE65']=0

      # output['ACS_PCT_AGE_15_17'].append(0)
      # output['ACS_PCT_AGE_0_17'].append(100)
      # output['ACS_PCT_AGE_30_44'].append(0)
      # output['ACS_PCT_AGE_45_64'].append(0)
      # output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<44 and input['age']>30):

      #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      output['ACS_PCT_AGE_15_17']=0
      output['ACS_PCT_AGE_0_17']=0
      output['ACS_PCT_AGE_30_44']=0
      output['ACS_PCT_AGE_45_64']=0
      output['ACS_PCT_AGE_ABOVE65']=0

      # output['ACS_PCT_AGE_15_17'].append(0)
      # output['ACS_PCT_AGE_0_17'].append(0)
      # output['ACS_PCT_AGE_30_44'].append(100)
      # output['ACS_PCT_AGE_45_64'].append(0)
      # output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']<=64 and input['age']>45):

      #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      output['ACS_PCT_AGE_15_17']=0
      output['ACS_PCT_AGE_0_17']=0
      output['ACS_PCT_AGE_30_44']=0
      output['ACS_PCT_AGE_45_64']=0
      output['ACS_PCT_AGE_ABOVE65']=0

      # output['ACS_PCT_AGE_15_17'].append(0)
      # output['ACS_PCT_AGE_0_17'].append(0)
      # output['ACS_PCT_AGE_30_44'].append(0)
      # output['ACS_PCT_AGE_45_64'].append(100)
      # output['ACS_PCT_AGE_ABOVE65'].append(0)

  elif(input['age']>=65):

      #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      output['ACS_PCT_AGE_15_17']=0
      output['ACS_PCT_AGE_0_17']=0
      output['ACS_PCT_AGE_30_44']=0
      output['ACS_PCT_AGE_45_64']=0
      output['ACS_PCT_AGE_ABOVE65']=0

      # output['ACS_PCT_AGE_15_17'].append(0)
      # output['ACS_PCT_AGE_0_17'].append(0)
      # output['ACS_PCT_AGE_30_44'].append(0)
      # output['ACS_PCT_AGE_45_64'].append(0)
      # output['ACS_PCT_AGE_ABOVE65'].append(100)

  else:
    output['ACS_PCT_AGE_15_17']=0
    output['ACS_PCT_AGE_0_17']=0
    output['ACS_PCT_AGE_30_44']=0
    output['ACS_PCT_AGE_45_64']=0
    output['ACS_PCT_AGE_ABOVE65']=0

    # output['ACS_PCT_AGE_15_17'].append(0)
    # output['ACS_PCT_AGE_0_17'].append(0)
    # output['ACS_PCT_AGE_30_44'].append(0)
    # output['ACS_PCT_AGE_45_64'].append(0)
    # output['ACS_PCT_AGE_ABOVE65'].append(0)

  return output

In [ ]:
def race_pct_multiply(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices):
  race=input['Race']
  row_ind=cencus_zip.index[cencus_zip['cencus_tract'] == input['Address']][0]


  if(input['Race']=='ASIAN'):

    asian=[]
    for index in indices:
      asian.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN'])

    cencus_zip['ACS_PCT_ASIAN']=asian


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_ASIAN']/100

      output[feat]=val
      #(output[feat]).append(val)
  elif(input['Race']=='AIAN'):

    asian2=[]
    for index in indices:
      asian2.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN'])

    cencus_zip['ACS_PCT_AIAN']=asian2


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_AIAN']/100

      output[feat]=val
      #(output[feat]).append(val)

  elif(input['Race']=='BLACK'):

    asian_male=[]
    for index in indices:
      asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK'])

    cencus_zip['ACS_PCT_BLACK']=asian_male


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_BLACK']/100

      output[feat]=val
      #(output[feat]).append(val)


  elif(input['Race']=='WHITE'):
    print('white female----')
    asian_male=[]
    for index in indices:
      asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE'])

    cencus_zip['ACS_PCT_WHITE']=asian_male


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_WHITE']/100

      output[feat]=val
      #(output[feat]).append(val)



  elif(input['Race']=='MULT'):

    asian_male=[]
    for index in indices:
      asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE'])

    cencus_zip['ACS_PCT_MULT_RACE']=asian_male


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_MULT_RACE']/100

      output[feat]=val
      #(output[feat]).append(val)

  elif(input['Race']=='OTHER'):

    asian_male=[]
    for index in indices:
      asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER'])

    cencus_zip['ACS_PCT_OTHER']=asian_male


    for feat in output.keys(): #for all features
      #keep the age columns 1 hot encoding at the last
      val=output[feat]*cencus_zip.loc[row_ind,'ACS_PCT_OTHER']/100

      output[feat]=val
      #(output[feat]).append(val)

  print('the dictionary-- from race multiply-----')
  print(output)
  print('-------------------------------')
  return output


In [ ]:
def main(input,df_60,ord_cencus_all):
  #Obtain the cencus_zip dataframe which contains the cencus tracts belonging to the same zipcode in the input and the total population column


  flag_dict=check_flag(input)
  print(flag_dict)

  if(input['zipcode']!=None):

    #cencus_zip,top_60_feature_only,total_population,indices
    cencus_zip,top_60_feature_only,total_population,indices=find_cencus(input,df_60,ord_cencus_all)

    if(input['Address']==None):
      address=cencus_zip.loc[0,'cencus_tract']
      state=cencus_zip.loc[0,'STATE']
      county=cencus_zip.loc[0,'COUNTY']



    output={col: [] for col in top_60_feature_only.columns}
    print(output)
    #if nothing is given
    if(flag_dict['flag_add']!=True and flag_dict['flag_gen']!=True and flag_dict['flag_age']!=True and flag_dict['flag_race']!=True and flag_dict['flag_income']!=True and flag_dict['flag_edu']!=True and flag_dict['flag_vet']!=True):

      print('only zip given')
      output=normal_impute(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output)
      #when address not given return the 1st cencus_tract from the cencus_zip
      #address=cencus_zip.loc[0,'cencus_tract'] #1st address
      return output,address,state,county

    elif(flag_dict['flag_add']==True):
      #when address given, store the address to return
      address=input['Address']
      state=cencus_zip.loc[0,'STATE']
      county=cencus_zip.loc[0,'COUNTY']

      print('address given only')
      #considering address=cencus_tract, only address given
      row=cencus_zip.loc[cencus_zip['cencus_tract']==input['Address']]
      col=row.keys()
      col_bad=col[:8]
      row_feature_only=row.drop(columns=col_bad)
      output = row_feature_only.to_dict('records')[0]
      print(output.keys())
      output.pop('population')
      print('the address extracted useful columns-----------')
      print(output)
      print('-------------------------------------')

      if(input['Race']!=None):
        output=race_pct_multiply(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      if(input['age']!=None):
        output=onehot_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)

      if(input['Gender']!=None):
        if(input['Gender']=='M'):
          output['ACS_PCT_MALE']=100

        elif(input['Gender']=='F'):
          output['ACS_PCT_MALE']=0

      new_output = {key: [value] for key, value in output.items()}
      return output,address,state,county#till now return, later see to remove

    elif(flag_dict['flag_add']!=True and flag_dict['flag_gen']==True and flag_dict['flag_age']!=True and flag_dict['flag_race']!=True and flag_dict['flag_income']!=True and flag_dict['flag_edu']!=True ):
      #only gendergiven
      #and flag_dict['flag_vet']!=True

      print('only gender given')
      output,cencus_zip_gend=impute_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      #address=cencus_zip.loc[0,'cencus_tract']

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      return output,address,state,county

    elif(flag_dict['flag_add']!=True  and flag_dict['flag_age']==True and flag_dict['flag_race']!=True and flag_dict['flag_income']!=True and flag_dict['flag_edu']!=True ):
      '''and flag_dict['flag_gen']!=None or None'''
      #age given gender can or not be given
      #and flag_dict['flag_vet']!=True
      print('gender given, age given only')
      if(flag_dict['flag_gen']==True):
        cencus_zip_gend=make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)

      print('age given: ',input['age'])
      output,cencus_zip=impute_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices) #give the cencus_zip_gend

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      return output,address,state,county



    elif(flag_dict['flag_add']!=True  and flag_dict['flag_race']==True and flag_dict['flag_income']!=True and flag_dict['flag_edu']!=True):
      #race given whether gender given or not
      # and flag_dict['flag_age']!=True
      #and flag_dict['flag_vet']!=True
      cencus_zip_gend=make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      #if(input['Race']==None): #take approximation of only 1 hot encoding of the age column
      cencus_zip_inc_gend=give_age_gender_income_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices)

      print('race given, age manager only')
      if(input['age']==None):#No age given
        output=race_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices)
        print('in the main function, age=None race=given------------')
        print(output)
        print('-----------------------------')

        if(input['Veteran Status']=='YES'):
          output['ACS_PCT_VET']=100
        elif(input['Veteran Status']!='YES'):
          output['ACS_PCT_VET']=0

        return output,address,state,county

      if(input['age']!=None):
        output=race_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
        print('in the main function------------')
        print(output)
        print('-----------------------------')
        if(input['age']<17 and input['age']>15):

            #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
            output['ACS_PCT_AGE_15_17'].clear()
            output['ACS_PCT_AGE_0_17'].clear()
            output['ACS_PCT_AGE_30_44'].clear()
            output['ACS_PCT_AGE_45_64'].clear()
            output['ACS_PCT_AGE_ABOVE65'].clear()

            output['ACS_PCT_AGE_15_17'].append(100)
            output['ACS_PCT_AGE_0_17'].append(0)
            output['ACS_PCT_AGE_30_44'].append(0)
            output['ACS_PCT_AGE_45_64'].append(0)
            output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<17):

            #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
            output['ACS_PCT_AGE_15_17'].clear()
            output['ACS_PCT_AGE_0_17'].clear()
            output['ACS_PCT_AGE_30_44'].clear()
            output['ACS_PCT_AGE_45_64'].clear()
            output['ACS_PCT_AGE_ABOVE65'].clear()

            output['ACS_PCT_AGE_15_17'].append(0)
            output['ACS_PCT_AGE_0_17'].append(100)
            output['ACS_PCT_AGE_30_44'].append(0)
            output['ACS_PCT_AGE_45_64'].append(0)
            output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<44 and input['age']>30):

            #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
            output['ACS_PCT_AGE_15_17'].clear()
            output['ACS_PCT_AGE_0_17'].clear()
            output['ACS_PCT_AGE_30_44'].clear()
            output['ACS_PCT_AGE_45_64'].clear()
            output['ACS_PCT_AGE_ABOVE65'].clear()

            output['ACS_PCT_AGE_15_17'].append(0)
            output['ACS_PCT_AGE_0_17'].append(0)
            output['ACS_PCT_AGE_30_44'].append(100)
            output['ACS_PCT_AGE_45_64'].append(0)
            output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<=64 and input['age']>45):

            #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
            output['ACS_PCT_AGE_15_17'].clear()
            output['ACS_PCT_AGE_0_17'].clear()
            output['ACS_PCT_AGE_30_44'].clear()
            output['ACS_PCT_AGE_45_64'].clear()
            output['ACS_PCT_AGE_ABOVE65'].clear()

            output['ACS_PCT_AGE_15_17'].append(0)
            output['ACS_PCT_AGE_0_17'].append(0)
            output['ACS_PCT_AGE_30_44'].append(0)
            output['ACS_PCT_AGE_45_64'].append(100)
            output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']>=65):

            #output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
            output['ACS_PCT_AGE_15_17'].clear()
            output['ACS_PCT_AGE_0_17'].clear()
            output['ACS_PCT_AGE_30_44'].clear()
            output['ACS_PCT_AGE_45_64'].clear()
            output['ACS_PCT_AGE_ABOVE65'].clear()

            output['ACS_PCT_AGE_15_17'].append(0)
            output['ACS_PCT_AGE_0_17'].append(0)
            output['ACS_PCT_AGE_30_44'].append(0)
            output['ACS_PCT_AGE_45_64'].append(0)
            output['ACS_PCT_AGE_ABOVE65'].append(100)

        else:
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)


        print('in the main function just before returning------------')
        print(output)
        print('-----------------------------')

        if(input['Veteran Status']=='YES'):
          output['ACS_PCT_VET']=100
        elif(input['Veteran Status']!='YES'):
          output['ACS_PCT_VET']=0

        if(input['Address']==None):
          return output,address,state,county
        else:
          return output

    elif(flag_dict['flag_add']!=True   and flag_dict['flag_income']==True and flag_dict['flag_edu']!=True ):
      print('income given,age manager,gender manager')
      #and flag_dict['flag_race']!=True
      #and flag_dict['flag_vet']!=True
      #income given is given
      #columns can be used
      #consider the age
      cencus_zip_gend=make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      #if(input['Race']==None): #take approximation of only 1 hot encoding of the age column
      cencus_zip_inc_gend=give_age_gender_income_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices)

      if(input['age']==None):
        output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
        print('ageless income---------')
        print(output)
        print('ageless income---------')
      elif(input['age']!=None):
        if(input['age']<17 and input['age']>15):
          print('age_1 income--------')
          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          print('age_1 income--------')
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(100)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<17):

          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(100)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<44 and input['age']>30):

          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(100)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<=64 and input['age']>45):

          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(100)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']>=65):

          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(100)

        else:
          print('diff age---------')
          output=imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
          print(output)
          print('special column------')
          print(output['ACS_PCT_AGE_15_17'])
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      return output,address,state,county


    elif(flag_dict['flag_add']!=True  and flag_dict['flag_income']==True and flag_dict['flag_edu']==True):
      #and flag_dict['flag_vet']!=True
      #education given
      #gender given
      #age given
      #income given    make it race independent
      # and flag_dict['flag_race']!=True
      print('edu given,gender_manager,age given,income given only')
      cencus_zip_gend=make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      cencus_zip_inc_gend=give_age_gender_income_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices)
      cencus_zip=give_edu_vet_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)


      output=impute_edu(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      return output,address,state,county

    elif(flag_dict['flag_add']!=True and flag_dict['flag_income']!=True and flag_dict['flag_edu']==True ):
      #and flag_dict['flag_vet']!=True
      #education given
      #gender given or not
      #age given or not
      #race_given or not
      # and flag_dict['flag_race']!=True
      print('edu given gender manager age manager race manager')
      education=input['Education']
      cencus_zip_gend=make_cencus_zip_gend(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
      cencus_zip_inc_gend=give_age_gender_income_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices)
      cencus_zip=give_edu_vet_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices)
      age_feat=''
      if(input['Race']!=None and input['age']==None):
        print('got race no age')
        output=income_less_edu_ageless(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education)
      elif(input['Race']!=None and input['age']!=None):
        print('got both age and gender')

        age=input['age']

        if(input['age']<17 and input['age']>15):

          age_feat='ACS_PCT_AGE_15_17'
          output=income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat)

          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(100)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<17):

          age_feat='ACS_PCT_AGE_0_17'
          output=income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(100)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<44 and input['age']>30):

          age_feat='ACS_PCT_AGE_30_44'
          output=income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(100)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']<=64 and input['age']>45):

          age_feat='ACS_PCT_AGE_45_64'
          output=income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(100)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

        elif(input['age']>=65):
          age_feat='ACS_PCT_AGE_ABOVE65'
          output=income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat)
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(100)

        else:
          print('diff age---------')
          output=income_less_edu_ageless(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)
          print(output)
          print('special column------')
          print(output['ACS_PCT_AGE_15_17'])
          output['ACS_PCT_AGE_15_17'].clear()
          output['ACS_PCT_AGE_0_17'].clear()
          output['ACS_PCT_AGE_30_44'].clear()
          output['ACS_PCT_AGE_45_64'].clear()
          output['ACS_PCT_AGE_ABOVE65'].clear()

          output['ACS_PCT_AGE_15_17'].append(0)
          output['ACS_PCT_AGE_0_17'].append(0)
          output['ACS_PCT_AGE_30_44'].append(0)
          output['ACS_PCT_AGE_45_64'].append(0)
          output['ACS_PCT_AGE_ABOVE65'].append(0)

      elif(input['Race']==None and input['age']==None):
        #depends only upon income and gender
        print('no age no race')
        output=income_less_edu_ageless(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education)

        if(input['Veteran Status']=='YES'):
          output['ACS_PCT_VET']=100
        elif(input['Veteran Status']!='YES'):
          output['ACS_PCT_VET']=0

        return output,address,state,county

      elif(input['Race']==None and input['age']!=None):
        print('no race got age')
        #impute_age returns the new cencus_zip dataframe and output dict
        output,cencus_age=impute_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices)

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      return output,address,state,county


    else:
      print('only default case')
      if(input['Veteran Status']!=None):
        veteran_pop_pct=[]
        for index in indices:
          veteran_pop_pct.append(ord_cencus_all.loc[index,'ACS_TOT_CIVIL_VET_POP_ABOVE25'])

        cencus_zip['ACS_TOT_CIVIL_VET_POP_ABOVE25']=veteran_pop_pct

        pop_deno=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            vet_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_TOT_CIVIL_VET_POP_ABOVE25']

            val+=df_60.loc[i,feat]*vet_pop
            pop_deno+=vet_pop

          val/=pop_deno
          (output[feat]).append(val)

      else:
        output=normal_impute(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,education)

      if(input['Veteran Status']=='YES'):
        output['ACS_PCT_VET']=100
      elif(input['Veteran Status']!='YES'):
        output['ACS_PCT_VET']=0

      if(input['Address']==None):
        return output,address,state,county
      else:
        return output,address,state,county


  else:
    print('give zipcode at least')
    return None

In [ ]:
def income_less_edu_aged(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education,age_feat):
  #race,gender,education is given
  #age is given
  if(education=='BACHELOR'):
    education='ACS_PCT_BACHELOR_DGR'
  elif(education=='COLLEGE_ASSOCIATE'):
    education='ACS_PCT_COLLEGE_ASSOCIATE_DGR'
  elif(education=='GRADUATE'):
    education='ACS_PCT_GRADUATE_DGR'
  elif(education=='HS_GRADUATE'):
    education='ACS_PCT_HS_GRADUATE'
  elif(education=='LT_HS'):
    education='ACS_PCT_LT_HS'
  elif(education=='POSTHS'):
    education='ACS_PCT_POSTHS_ED'


  if(input['Gender']=='M'):
    if(input['Race']=='ASIAN'):
      print('asian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_MALE'])

      cencus_zip['ACS_PCT_ASIAN_MALE']=asian_male

      #remove the age=None column


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='AIAN'):
      print('aian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_MALE'])

      cencus_zip['ACS_PCT_AIAN_MALE']=asian_male

      #remove the age=None column


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=cencus_zip.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='BLACK'):
      print('black male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_MALE'])

      cencus_zip['ACS_PCT_BLACK_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='WHITE'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_MALE'])

      cencus_zip['ACS_PCT_WHITE_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='MULT'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_MALE'])

      cencus_zip['ACS_PCT_MULT_RACE_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='OTHER'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_MALE'])

      cencus_zip['ACS_PCT_OTHER_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_MALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Gender']=='F'):
      if(input['Race']=='ASIAN'):

        asian_female=[]
        for index in indices:
          asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_FEMALE'])

        cencus_zip['ACS_PCT_ASIAN_FEMALE']=asian_female


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_female_pop
            pop_deno_race+=asian_female_pop

          val/=pop_deno_race
          (output[feat]).append(val)

      elif(input['Race']=='AIAN'):

        asian_female=[]
        for index in indices:
          asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_FEMALE'])

        cencus_zip['ACS_PCT_AIAN_FEMALE']=asian_female


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_female_pop
            pop_deno_race+=asian_female_pop

          val/=pop_deno_race
          (output[feat]).append(val)

      elif(input['Race']=='BLACK'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_FEMALE'])

        cencus_zip['ACS_PCT_BLACK_FEMALE']=asian_male


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)


      elif(input['Race']=='WHITE'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_FEMALE'])

        cencus_zip['ACS_PCT_WHITE_FEMALE']=asian_male


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)



      elif(input['Race']=='MULT'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_FEMALE'])

        cencus_zip['ACS_PCT_MULT_RACE_FEMALE']=asian_male


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

      elif(input['Race']=='OTHER'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_FEMALE'])

        cencus_zip['ACS_PCT_OTHER_FEMALE']=asian_male


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_FEMALE']*cencus_zip.loc[i,education]*cencus_zip.loc[i,age_feat]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

  return output


In [ ]:
def income_less_edu_ageless(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices,education):
  #race,gender,education is given
  #age is not given
  if(education=='BACHELOR'):
    education='ACS_PCT_BACHELOR_DGR'
  elif(education=='COLLEGE_ASSOCIATE'):
    education='ACS_PCT_COLLEGE_ASSOCIATE_DGR'
  elif(education=='GRADUATE'):
    education='ACS_PCT_GRADUATE_DGR'
  elif(education=='HS_GRADUATE'):
    education='ACS_PCT_HS_GRADUATE'
  elif(education=='LT_HS'):
    education='ACS_PCT_LT_HS'
  elif(education=='POSTHS'):
    education='ACS_PCT_POSTHS_ED'


  if(input['Gender']=='M'):
    if(input['Race']=='ASIAN'):
      print('asian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_MALE'])

      cencus_zip['ACS_PCT_ASIAN_MALE']=asian_male

      if(input['age']==None):


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

    elif(input['Race']=='AIAN'):
      print('asian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_MALE'])

      cencus_zip['ACS_PCT_AIAN_MALE']=asian_male
      print(cencus_zip['ACS_PCT_AIAN_MALE'].isnull().sum())
      if(input['age']==None):


        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

    elif(input['Race']=='BLACK'):
      print('black male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_MALE'])

      cencus_zip['ACS_PCT_BLACK_MALE']=asian_male

      if(input['age']==None):
        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)


    elif(input['Race']=='WHITE'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_MALE'])

      cencus_zip['ACS_PCT_WHITE_MALE']=asian_male

      if(input['age']==None):
        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)


    elif(input['Race']=='MULT'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_MALE'])

      cencus_zip['ACS_PCT_MULT_RACE_MALE']=asian_male

      if(input['age']==None):
        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

    elif(input['Race']=='OTHER'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_MALE'])

      cencus_zip['ACS_PCT_OTHER_MALE']=asian_male

      if(input['age']==None):
        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

    elif (input['Race']==None):
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_MALE'])

      cencus_zip['ACS_PCT_OTHER_MALE']=asian_male

      if(input['age']==None):
        pop_deno_race=0
        for feat in top_60_feature_only.keys(): #for all features
          val=0
          for i in range(cencus_zip.shape[0]):#for all the cencus
            asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_MALE']*cencus_zip.loc[i,education]

            val+=df_60.loc[i,feat]*asian_male_pop
            pop_deno_race+=asian_male_pop

          val/=pop_deno_race
          (output[feat]).append(val)

    elif(input['Gender']=='F'):
      if(input['Race']=='ASIAN'):

        asian_female=[]
        for index in indices:
          asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_FEMALE'])

        cencus_zip['ACS_PCT_ASIAN_FEMALE']=asian_female

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_female_pop
              pop_deno_race+=asian_female_pop

            val/=pop_deno_race
            (output[feat]).append(val)

      if(input['Race']=='AIAN'):

        asian_female=[]
        for index in indices:
          asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_FEMALE'])

        cencus_zip['ACS_PCT_AIAN_FEMALE']=asian_female

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_female_pop
              pop_deno_race+=asian_female_pop

            val/=pop_deno_race
            (output[feat]).append(val)


      elif(input['Race']=='BLACK'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_FEMALE'])

        cencus_zip['ACS_PCT_BLACK_FEMALE']=asian_male

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_male_pop
              pop_deno_race+=asian_male_pop

            val/=pop_deno_race
            (output[feat]).append(val)


      elif(input['Race']=='WHITE'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_FEMALE'])

        cencus_zip['ACS_PCT_WHITE_FEMALE']=asian_male

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_male_pop
              pop_deno_race+=asian_male_pop

            val/=pop_deno_race
            (output[feat]).append(val)



      elif(input['Race']=='MULT'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_FEMALE'])

        cencus_zip['ACS_PCT_MULT_RACE_FEMALE']=asian_male

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_male_pop
              pop_deno_race+=asian_male_pop

            val/=pop_deno_race
            (output[feat]).append(val)

      elif(input['Race']=='OTHER'):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_FEMALE'])

        cencus_zip['ACS_PCT_OTHER_FEMALE']=asian_male

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_male_pop
              pop_deno_race+=asian_male_pop

            val/=pop_deno_race
            (output[feat]).append(val)

      elif(input['Race']==None):

        asian_male=[]
        for index in indices:
          asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_FEMALE'])

        cencus_zip['ACS_PCT_OTHER_FEMALE']=asian_male

        if(input['age']==None):
          pop_deno_race=0
          for feat in top_60_feature_only.keys(): #for all features
            val=0
            for i in range(cencus_zip.shape[0]):#for all the cencus
              asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_FEMALE']*cencus_zip.loc[i,education]

              val+=df_60.loc[i,feat]*asian_male_pop
              pop_deno_race+=asian_male_pop

            val/=pop_deno_race
            (output[feat]).append(val)

  return output

In [ ]:
def impute_edu(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices):
  #handles both age given or not given
  #gender,income,education given
  if(input['age']==None):
    output=imput_gender_income_edu(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'])

  else:
    age_feat=''
    #output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],input['age'])
    age=input['age']

    if(age>15 and age<=17):
      age_feat='ACS_PCT_AGE_15_17'
      output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],age_feat)
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(100)
      output['ACS_PCT_AGE_0_17'].append(0)
      output['ACS_PCT_AGE_30_44'].append(0)
      output['ACS_PCT_AGE_45_64'].append(0)
      output['ACS_PCT_AGE_ABOVE65'].append(0)

    elif(age<17):
      age_feat='ACS_PCT_AGE_0_17'
      output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],age_feat)
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(0)
      output['ACS_PCT_AGE_0_17'].append(100)
      output['ACS_PCT_AGE_30_44'].append(0)
      output['ACS_PCT_AGE_45_64'].append(0)
      output['ACS_PCT_AGE_ABOVE65'].append(0)

    elif(age>30 and age<=44):
      age_feat='ACS_PCT_AGE_30_44'
      output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],age_feat)
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(0)
      output['ACS_PCT_AGE_0_17'].append(0)
      output['ACS_PCT_AGE_30_44'].append(100)
      output['ACS_PCT_AGE_45_64'].append(0)
      output['ACS_PCT_AGE_ABOVE65'].append(0)

    elif(age>45 and age<=64):
      age_feat='ACS_PCT_AGE_45_64'
      output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],age_feat)
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(0)
      output['ACS_PCT_AGE_0_17'].append(0)
      output['ACS_PCT_AGE_30_44'].append(0)
      output['ACS_PCT_AGE_45_64'].append(100)
      output['ACS_PCT_AGE_ABOVE65'].append(0)

    elif(age>=65):
      age_feat='ACS_PCT_AGE_ABOVE65'
      output=imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'],age_feat)
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(0)
      output['ACS_PCT_AGE_0_17'].append(0)
      output['ACS_PCT_AGE_30_44'].append(0)
      output['ACS_PCT_AGE_45_64'].append(0)
      output['ACS_PCT_AGE_ABOVE65'].append(100)
    else:
      output=imput_gender_income_edu(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,input['Education'])
      output['ACS_PCT_AGE_15_17'].clear()
      output['ACS_PCT_AGE_0_17'].clear()
      output['ACS_PCT_AGE_30_44'].clear()
      output['ACS_PCT_AGE_45_64'].clear()
      output['ACS_PCT_AGE_ABOVE65'].clear()

      output['ACS_PCT_AGE_15_17'].append(0)
      output['ACS_PCT_AGE_0_17'].append(0)
      output['ACS_PCT_AGE_30_44'].append(0)
      output['ACS_PCT_AGE_45_64'].append(0)
      output['ACS_PCT_AGE_ABOVE65'].append(0)

  if(input['Education']=='LT_HS'):
      output['ACS_PCT_LT_HS']=100
  else:
    output['ACS_PCT_LT_HS']=0

  return output

In [ ]:
def imput_gender_income_edu_age(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,education_feature,age_feat):
  #gender income education age given
  #one hot encoding of age is done later
  if(education_feature=='BACHELOR'):
    education_feature='ACS_PCT_BACHELOR_DGR'
  elif(education_feature=='COLLEGE_ASSOCIATE'):
    education_feature='ACS_PCT_COLLEGE_ASSOCIATE_DGR'
  elif(education_feature=='GRADUATE'):
    education_feature='ACS_PCT_GRADUATE_DGR'
  elif(education_feature=='HS_GRADUATE'):
    education_feature='ACS_PCT_HS_GRADUATE'
  elif(education_feature=='LT_HS'):
    education_feature='ACS_PCT_LT_HS'
  elif(education_feature=='POSTHS'):
    education_feature='ACS_PCT_POSTHS_ED'

  #impute gender income along with education
  if(input['Gender']==None):

    if(input['Income']<10000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<14999 and input['Income']>=10000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<24999 and input['Income']>=15000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<49999 and input['Income']>=25000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<99999 and input['Income']>=49999):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']>=100000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

  elif(input['Gender']!=None):
    if(input['Gender']=='M'):
          print('incoming launda')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']>=100000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

    elif(input['Gender']=='F'): #laundioke liye
          print('incoming laundi')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            print('dhoni')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            print('borolok')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)
            print('the upcoming dictionary-----------')


          elif(input['Income']>=100000):
            print('poisa')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]*cencus_zip_inc_gend.loc[i,age_feat]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

  return output

In [ ]:
def imput_gender_income_edu(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices,education_feature):

  if(education_feature=='BACHELOR'):
    education_feature='ACS_PCT_BACHELOR_DGR'
  elif(education_feature=='COLLEGE_ASSOCIATE'):
    education_feature='ACS_PCT_COLLEGE_ASSOCIATE_DGR'
  elif(education_feature=='GRADUATE'):
    education_feature='ACS_PCT_GRADUATE_DGR'
  elif(education_feature=='HS_GRADUATE'):
    education_feature='ACS_PCT_HS_GRADUATE'
  elif(education_feature=='LT_HS'):
    education_feature='ACS_PCT_LT_HS'
  elif(education_feature=='POSTHS'):
    education_feature='ACS_PCT_POSTHS_ED'

  #impute gender income along with education
  if(input['Gender']==None):

    if(input['Income']<10000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<14999 and input['Income']>=10000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<24999 and input['Income']>=15000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<49999 and input['Income']>=25000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']<99999 and input['Income']>=49999):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

    elif(input['Income']>=100000):
      pop_deno=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
          inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,education_feature]

          val+=df_60.loc[i,feat]*inc_pop
          pop_deno+=inc_pop

        val/=pop_deno
        (output[feat]).append(val)

  elif(input['Gender']!=None):
    if(input['Gender']=='M'):
          print('incoming launda')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']>=100000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'male_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

    elif(input['Gender']=='F'): #laundioke liye
          print('incoming laundi')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            print('dhoni')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            print('borolok')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)
            print('the upcoming dictionary-----------')


          elif(input['Income']>=100000):
            print('poisa')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'female_percent']*cencus_zip_inc_gend.loc[i,education_feature]

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

  return output

In [ ]:
def imput_gender_income(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_inc_gend,total_population,output,indices):
  if(input['Gender']==None):

              if(input['Income']<10000):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

              elif(input['Income']<14999 and input['Income']>=10000):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

              elif(input['Income']<24999 and input['Income']>=15000):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

              elif(input['Income']<49999 and input['Income']>=25000):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

              elif(input['Income']<99999 and input['Income']>=49999):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

              elif(input['Income']>=100000):
                pop_deno=0
                for feat in top_60_feature_only.keys(): #for all features
                  val=0
                  for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                    inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']

                    val+=df_60.loc[i,feat]*inc_pop
                    pop_deno+=inc_pop

                  val/=pop_deno
                  (output[feat]).append(val)

  elif(input['Gender']!=None):
    if(input['Gender']=='M'):
          print('incoming launda')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']>=100000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'male_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

    elif(input['Gender']=='F'): #laundioke liye
          print('incoming laundi')
          if(input['Income']<10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_10000']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<14999 and input['Income']>=10000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_14999']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<24999 and input['Income']>=15000):
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_24999']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<49999 and input['Income']>=25000):
            print('dhoni')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_49999']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

          elif(input['Income']<99999 and input['Income']>=49999):
            print('borolok')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_99999']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)
            print('the upcoming dictionary-----------')


          elif(input['Income']>=100000):
            print('poisa')
            pop_deno=0
            for feat in top_60_feature_only.keys(): #for all features
              val=0
              for i in range(cencus_zip_inc_gend.shape[0]):#for all the cencus
                inc_pop=cencus_zip_inc_gend.loc[i,'population']*cencus_zip_inc_gend.loc[i,'ACS_PCT_HH_INC_100000']*cencus_zip_inc_gend.loc[i,'female_percent']

                val+=df_60.loc[i,feat]*inc_pop
                pop_deno+=inc_pop

              val/=pop_deno
              (output[feat]).append(val)

  return output

In [ ]:
def race_gender(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip,total_population,output,indices):
  if(input['Gender']=='M'):
    if(input['Race']=='ASIAN'):
      print('asian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_MALE'])

      cencus_zip['ACS_PCT_ASIAN_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='AIAN'):
      print('asian male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_MALE'])

      cencus_zip['ACS_PCT_AIAN_MALE']=asian_male
      print(cencus_zip['ACS_PCT_AIAN_MALE'].isnull().sum())

      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='BLACK'):
      print('black male')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_MALE'])

      cencus_zip['ACS_PCT_BLACK_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='WHITE'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_MALE'])

      cencus_zip['ACS_PCT_WHITE_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='MULT'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_MALE'])

      cencus_zip['ACS_PCT_MULT_RACE_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='OTHER'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_MALE'])

      cencus_zip['ACS_PCT_OTHER_MALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_MALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

  elif(input['Gender']=='F'):
    if(input['Race']=='ASIAN'):

      asian_female=[]
      for index in indices:
        asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN_FEMALE'])

      cencus_zip['ACS_PCT_ASIAN_FEMALE']=asian_female


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN_FEMALE']

          val+=df_60.loc[i,feat]*asian_female_pop
          pop_deno_race+=asian_female_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='AIAN'):

      asian_female=[]
      for index in indices:
        asian_female.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN_FEMALE'])

      cencus_zip['ACS_PCT_AIAN_FEMALE']=asian_female


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN_FEMALE']

          val+=df_60.loc[i,feat]*asian_female_pop
          pop_deno_race+=asian_female_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='BLACK'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK_FEMALE'])

      cencus_zip['ACS_PCT_BLACK_FEMALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK_FEMALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='WHITE'):
      print('white female----')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE_FEMALE'])

      cencus_zip['ACS_PCT_WHITE_FEMALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE_FEMALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

      print('the dictionary-- from race_gender-----')
      print(output)
      print('-------------------------------')

    elif(input['Race']=='MULT'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE_FEMALE'])

      cencus_zip['ACS_PCT_MULT_RACE_FEMALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE_FEMALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='OTHER'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER_FEMALE'])

      cencus_zip['ACS_PCT_OTHER_FEMALE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER_FEMALE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


  elif(input['Gender']==None):
    if(input['Race']=='ASIAN'):

      asian=[]
      for index in indices:
        asian.append(ord_cencus_all.loc[index,'ACS_PCT_ASIAN'])

      cencus_zip['ACS_PCT_ASIAN']=asian


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_ASIAN']

          val+=df_60.loc[i,feat]*asian_female_pop
          pop_deno_race+=asian_female_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='AIAN'):

      asian=[]
      for index in indices:
        asian.append(ord_cencus_all.loc[index,'ACS_PCT_AIAN'])

      cencus_zip['ACS_PCT_AIAN']=asian


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_female_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_AIAN']

          val+=df_60.loc[i,feat]*asian_female_pop
          pop_deno_race+=asian_female_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='BLACK'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_BLACK'])

      cencus_zip['ACS_PCT_BLACK']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_BLACK']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)


    elif(input['Race']=='WHITE'):
      print('white female----')
      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_WHITE'])

      cencus_zip['ACS_PCT_WHITE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_WHITE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

      print('the dictionary-- from race_gender-----')
      print(output)
      print('-------------------------------')

    elif(input['Race']=='MULT'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_MULT_RACE'])

      cencus_zip['ACS_PCT_MULT_RACE']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_MULT_RACE']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

    elif(input['Race']=='OTHER'):

      asian_male=[]
      for index in indices:
        asian_male.append(ord_cencus_all.loc[index,'ACS_PCT_OTHER'])

      cencus_zip['ACS_PCT_OTHER']=asian_male


      pop_deno_race=0
      for feat in top_60_feature_only.keys(): #for all features
        val=0
        for i in range(cencus_zip.shape[0]):#for all the cencus
          asian_male_pop=cencus_zip.loc[i,'population']*cencus_zip.loc[i,'ACS_PCT_OTHER']

          val+=df_60.loc[i,feat]*asian_male_pop
          pop_deno_race+=asian_male_pop

        val/=pop_deno_race
        (output[feat]).append(val)

  return output

In [ ]:
def give_edu_vet_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices):
  #assume having the previous 2 cencus zip versions
  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_BACHELOR_DGR'])

  cencus_zip_gend['ACS_PCT_BACHELOR_DGR']=edu_specific_col

  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_COLLEGE_ASSOCIATE_DGR'])

  cencus_zip_gend['ACS_PCT_COLLEGE_ASSOCIATE_DGR']=edu_specific_col

  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_GRADUATE_DGR'])

  cencus_zip_gend['ACS_PCT_GRADUATE_DGR']=edu_specific_col

  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_HS_GRADUATE'])

  cencus_zip_gend['ACS_PCT_HS_GRADUATE']=edu_specific_col

  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_LT_HS'])

  cencus_zip_gend['ACS_PCT_LT_HS']=edu_specific_col

  edu_specific_col=[]
  for index in indices:
      edu_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_POSTHS_ED'])

  cencus_zip_gend['ACS_PCT_POSTHS_ED']=edu_specific_col

#veteran status included
  edu_vet_specific_col=[]
  for index in indices:
      edu_vet_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_VET_BACHELOR'])

  cencus_zip_gend['ACS_PCT_VET_BACHELOR']=edu_vet_specific_col

  edu_vet_specific_col=[]
  for index in indices:
      edu_vet_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_VET_COLLEGE'])

  cencus_zip_gend['ACS_PCT_VET_COLLEGE']=edu_vet_specific_col

  edu_vet_specific_col=[]
  for index in indices:
      edu_vet_specific_col.append(ord_cencus_all.loc[index,'ACS_PCT_VET_HS'])

  cencus_zip_gend['ACS_PCT_VET_HS']=edu_vet_specific_col

  return cencus_zip_gend

In [ ]:
def give_age_gender_income_cencus_zip(input,df_60,top_60_feature_only,ord_cencus_all,cencus_zip_gend,total_population,output,indices):
  #Add age specific columns
  age_specific_col=[]
  for index in indices:
      age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_15_17'])

  cencus_zip_gend['ACS_PCT_AGE_15_17']=age_specific_col

  age_specific_col=[]
  for index in indices:
      age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_0_17'])

  cencus_zip_gend['ACS_PCT_AGE_0_17']=age_specific_col

  age_specific_col=[]
  for index in indices:
      age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_0_17'])

  cencus_zip_gend['ACS_PCT_AGE_30_44']=age_specific_col

  age_specific_col=[]
  for index in indices:
      age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_0_17'])

  cencus_zip_gend['ACS_PCT_AGE_45_64']=age_specific_col

  age_specific_col=[]
  for index in indices:
      age_specific_col.append(df_60.loc[index,'ACS_PCT_AGE_ABOVE65'])

  cencus_zip_gend['ACS_PCT_AGE_ABOVE65']=age_specific_col


#the income_specific columns added
  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_10000'])

  print('cencus_zip_gend shape',cencus_zip_gend.shape)
  print(len(income_specific_column))

  cencus_zip_gend['ACS_PCT_HH_INC_10000']=income_specific_column

  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_14999'])

  cencus_zip_gend['ACS_PCT_HH_INC_14999']=income_specific_column

  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_24999'])

  cencus_zip_gend['ACS_PCT_HH_INC_24999']=income_specific_column

  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_49999'])

  cencus_zip_gend['ACS_PCT_HH_INC_49999']=income_specific_column

  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_99999'])

  cencus_zip_gend['ACS_PCT_HH_INC_99999']=income_specific_column

  income_specific_column=[]
  for index in indices:
    income_specific_column.append(ord_cencus_all.loc[index,'ACS_PCT_HH_INC_100000'])

  cencus_zip_gend['ACS_PCT_HH_INC_100000']=income_specific_column

  #gender specific income columns:--
  gend_income_specific_column=[]
  for index in indices:
    gend_income_specific_column.append(ord_cencus_all.loc[index,'ACS_MEDIAN_INC_F'])

  cencus_zip_gend['ACS_MEDIAN_INC_F']=gend_income_specific_column

  gend_income_specific_column=[]
  for index in indices:
    gend_income_specific_column.append(ord_cencus_all.loc[index,'ACS_MEDIAN_INC_M'])

  cencus_zip_gend['ACS_MEDIAN_INC_M']=gend_income_specific_column


  return cencus_zip_gend

pandas.core.frame.DataFrame

#Gemma 2B

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN_READ')

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

# text = """<start_of_turn>user
# How does the brain work?<end_of_turn>
# <start_of_turn>model"""
# device = "cuda:0"
# inputs = tokenizer(text, return_tensors="pt").to(device)

# outputs = model.generate(**inputs, max_new_tokens=50)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ /Datasets/Workable Datasets/48_reduced/LLm_on_10k_train.csv')

In [ ]:
df=df.drop(['Unnamed: 0', 'YEAR','COUNTYFIPS', 'STATEFIPS','TERRITORY'], axis=1)

In [ ]:
df2=pd.read_csv("/content/Social.csv")

In [ ]:

# Create a dictionary to map codes to descriptions
code_to_description = dict(zip(df2['name'], df2['label']))
mapping = {col: code_to_description.get(col, col) for col in df2.columns}

# Rename columns while preserving order
df.rename(columns=mapping, inplace=True)
# Rename columns in df2
df.rename(columns=code_to_description, inplace=True)

# Print the result



In [ ]:
import pandas as pd

def combine_column_and_value(row):
    """ Combines a row's column names and values into a single string """
    result = " ".join([f"{col}: {str(value)}" for col, value in row.iteritems()])
    return result

# Apply the function to every row, creating a list of combined strings
new_data = df.apply(combine_column_and_value, axis=1).tolist()

# Create the new DataFrame
new_df = pd.DataFrame(new_data, columns=['combined'])



# Display the new DataFrame
new_df.columns

In [ ]:
new_df.combined[1]

In [ ]:
result_df=pd.DataFrame()

In [ ]:
data = new_df['combined']  # Adjust the column name if needed
result = data.str.split("Problem 1:", expand=True)
result_df['narrative'] = result[0]
result_df['response'] = result[1]


In [ ]:
result_df['response']="Problem 1:" + result_df['response']
result_df

In [ ]:
result_df['narrative']=result_df['narrative']+"You are a smart model, state the 3 most probable problems this person may face based on the information provided and also give solutions to the respective problems?(List exactly 3 problems and their respective solutions)(Try to be concise and to the point)"

In [ ]:
test_df=result_df.tail(2000)

In [ ]:
result_df = result_df.iloc[:-2000]

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(result_df[["narrative","response"]])

In [ ]:
dataset['narrative'][0]

In [ ]:
def formatting_func(example):
    text = f"<start_of_turn>user\n{example['narrative'][0]}<end_of_turn> <start_of_turn>model\n{example['response'][0]}<end_of_turn>"
    return [text]

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=300,
        per_device_train_batch_size=3,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained("/content/drive/MyDrive/ /Datasets/synthea data/Model_Gemma_2B_10k")
trainer.tokenizer.save_pretrained("/content/drive/MyDrive/ /Datasets/synthea data/Tokenizer_Gemma_2B_10k")

In [ ]:
model_gemma = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/ /Datasets/synthea data/Model_Gemma_2B_10k")
tokenizer_gemma = AutoTokenizer.from_pretrained("/content/drive/MyDrive/ /Datasets/synthea data/Tokenizer_Gemma_2B_10k")

In [ ]:
test_df=test_df.reset_index(drop=True)

In [ ]:
test_df

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

In [ ]:
text = """<start_of_turn>user
State-county-census tract FIPS Code (11-digit): 53053063301 State name: Washington County name: Pierce County Census region name: West Percentage of households with a smartphone with no other type of computing device: 5.4 Percentage of population between ages 15-17: 5.14 Percentage of population with employer-based and direct-purchase coverage: 0.58 Percentage of employed working in wholesale trade (ages 16 and over): 2.88 Percentage of households with other type of computer: 0.93 Percentage of male population now married and spouse absent (ages 15 and over): 6.21 Percentage of female population now married and spouse absent (ages 15 and over): 5.45 Percentage of employed working in other services, except public administration (ages 16 and over): 1.85 Percentage of civilian veterans that have some college education or an associate's degree (ages 25 and over): 27.14 Percentage of population who speak other Indo-European languages (ages 5 and over): 0.98 Percentage of population with other health insurance coverage combinations: 6.52 Percentage of population that moved within the same county in the past year (age 1 and over): 6.21 Percentage of employed working in finance and insurance, real estate, and rental and leasing (ages 16 and over): 1.13 75-84_e(x): 10.22 Percentage of civilian employed population working in public administration (ages 16 and over): 5.1 Percentage of population that moved from different county within same state in the past year (age 1 and over): 2.66 Percentage of employed working in transportation and warehousing, and in utilities (ages 16 and over): 4.37 Total grandchildren under 18 living with grandparent householder: 204.0 Percentage of employed working in construction (ages 16 and over): 13.23 Percentage of employed working in information services (ages 16 and over): 0.41 Percentage of households with broadband such as cable, fiber optic or DSL with no other type of Internet subscription: 2.93 Percentage of employed working in professional, scientific, management, administrative, and waste management services (ages 16 and over): 11.48 Percentage of employed working in arts, entertainment, recreation, and accommodation and food services (ages 16 and over): 22.03 Total population (between ages 16 and 19): 275.0 Percentage of population reporting multiple races: 4.46 Percentage of population with direct-purchase health insurance only: 4.09 Distance in miles to the nearest health clinic (FQHC, RHC), calculated using population weighted tract centroids: 1.31 Percentage of households with satellite internet service: 4.94 Distance in miles to the nearest emergency department, calculated using population weighted tract centroids: 2.41 Percentage of population who are private not-for-profit wage and salary workers (ages 16 and over): 13.9 Distance in miles to the nearest urgent care, calculated using population weighted tract centroids: 1.57 85 and older_e(x): 3.12 Distance in miles to the nearest designated trauma center, calculated using population weighted tract centroids: 2.41 Percentage of female population who worked in place of residence (ages 16 and over): 42.74 Percentage of population reporting Black or African American race alone: 25.72 Percentage of workers with at least 60-minute commute time (ages 16 and over): 14.03 Land area in square miles: 0.73 Percentage of employed working in manufacturing (ages 16 and over): 12.4 Percentage of occupied housing units with fuel oil heating: 0.0 Percentage of population consisting of U.S. citizens by naturalization: 5.39 OBESITY_Data_Value: 31.42 MAMMOUSE_Data_Value: 73.8 DEPRESSION_Data_Value: 23.24 CHOLSCREEN_Data_Value: 83.47999999999999 COLON_SCREEN_Data_Value: 69.67999999999999 Percentage of workers taking taxicab, motorcycle, bicycle, or other means to work (ages 16 and over): 1.41 Percentage of households with internet access without a subscription: 2.62 Percentage of children with a disability (ages 17 and below): 2.26 Percentage of population between ages 0-17: 26.03 Percentage of population between ages 30-44: 23.05 Percentage of population between ages 45-64: 23.46 Percentage of population ages 65 and over: 13.01 Percentage of population that is male: 56.18 Percentage of the civilian population consisting of veterans (ages 18 and over): 12.19 Per capita income (dollars, inflation-adjusted to data file year): 25042.0 Percentage of population with less than high school education (ages 25 and over): 14.78 Median gross rent (dollars): 1361.0 Percentage of households with only one occupant: 24.77 Percentage of population with no health insurance coverage: 15.53 .You are a smart model, state the 3 most probable problems this person may face based on the information provided and also give solutions to the respective problems?(List exactly 3 problems and their respective solutions)(Try to be concise and to the point)<end_of_turn>
<start_of_turn>model"""
inputs = tokenizer(text, return_tensors="pt")
# def check_and_clean_output(generated_text):
#     # Basic example using regular expressions
#     import re
#     sentences = re.split(r'[.!?\n]', generated_text)
#     complete_sentences = [s for s in sentences if s]  # Filter for non-empty sentences

#     # Remove any trailing asterisks
#     clean_output = "".join(s.rstrip('*') + "." for s in complete_sentences[:3])
#     return clean_output

outputs = model_gemma.generate(**inputs, max_new_tokens=300)
print(tokenizer_gemma.decode(outputs[0], skip_special_tokens=True))

#Bert

In [ ]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

In [ ]:
import pandas as pd
df=pd.read_csv('/content/filtered_health_data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Categories

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Category').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.data['label'] = self.label_encoder.fit_transform(self.data['Category'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx, 0])  # Convert to string explicitly
        label = self.data.iloc[idx, 2]  # Use the encoded label

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
num_categories = df['Category'].nunique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_categories)


In [ ]:
batch_size = 32
max_length = 128
epochs = 3
learning_rate = 2e-5

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {average_train_loss}')

In [ ]:
model.eval()
val_loss = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        val_loss += loss.item()

average_val_loss = val_loss / len(val_loader)
print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss}')

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Split the data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.data['label'] = self.label_encoder.fit_transform(self.data['Category'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx, 0])  # Convert to string explicitly
        label = self.data.iloc[idx, 2]  # Use the encoded label

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get the number of unique categories
num_categories = df['Category'].nunique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_categories)

# Define training parameters
batch_size = 32
max_length = 128
epochs = 3
learning_rate = 2e-5

# Create instances of the custom dataset
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()



In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {average_train_loss}')

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
            predicted_categories = train_dataset.label_encoder.inverse_transform(predicted_labels.tolist())
            # Print or store the predicted categories as needed
            print(predicted_categories)

            loss = loss_fn(logits, labels)
            val_loss += loss.item()

    average_val_loss = val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert')

In [ ]:
def predict_category(input_text): # Tokenize the input text
  inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True) # Pass the input through the model
  outputs = model(**inputs) # Get the predicted label ID
  predicted_label_id = torch.argmax(outputs.logits).item() # Decode the predicted label using the label encoder
  predicted_category = train_dataset.label_encoder.inverse_transform([predicted_label_id])[0]
  return predicted_category

In [ ]:
input_text = "ZIPCode: 90001, Income: $60000, Gender: Female.. SDOH Problems: Inadequate health insurance, Inadequate healthcare coverage; Long commute times affecting work-life balance"
predicted_category = predict_category(input_text)
print(predicted_category)

In [ ]:
# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert1')

In [ ]:
# prompt: save model on drive

!google-drive-upload fine_tuned_bert1/

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import joblib

# Assuming df is your DataFrame loaded with the data and the 'Category' column for labels

# Split the data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.data['label'] = self.label_encoder.fit_transform(self.data['Category'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx, 0])  # Assuming text is in the first column
        label = self.data.iloc[idx]['label']  # Use the encoded label

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get the number of unique categories
num_categories = df['Category'].nunique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_categories)

# Define training parameters
batch_size = 32
max_length = 128
epochs = 3
learning_rate = 2e-5

# Create instances of the custom dataset
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)



In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {average_train_loss}')

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = outputs.loss
            val_loss += loss.item()

    average_val_loss = val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss}')

# Save the fine-tuned model, tokenizer, and label encoder
model_save_path = 'fine_tuned_bert'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

model.save_pretrained(model_save_path)  # Save the model
tokenizer.save_pretrained(model_save_path)  # Save the tokenizer
label_encoder_save_path = os.path.join(model_save_path, 'label_encoder.pkl')
joblib.dump(train_dataset.label_encoder, label_encoder_save_path)  # Save the label encoder


In [ ]:
import torch

# Specify the path where you want to save the model
model_save_path = 'fine_tuned_bert_model.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), model_save_path)


#Mistral 7b

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/ /Datasets/10k_bad.csv")
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df

In [ ]:
def formatting_func(example):
    text = f"<s>[INST] {df['narrative']} [/INST] {df['response']}"
    return text

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["narrative","response"]])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [ ]:
new_dataset = dataset.map(generate_and_tokenize_prompt)

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

In [ ]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
import transformers
from datetime import datetime

project = "mistral_10k"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=new_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=2,
        output_dir=output_dir,
        per_device_train_batch_size=20,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_steps=25,                # Save checkpoints every 50 steps
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

In [ ]:
trainer.model.save_pretrained("model_mistral")
# trainer.tokenizer.save_pretrained("model_mistral")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "model_mistral")

In [ ]:
eval_prompt = """State-county-census tract FIPS Code (11-digit): 53053063301 State name: Washington County name: Pierce County Census region name: West Percentage of households with a smartphone with no other type of computing device: 5.4 Percentage of population between ages 15-17: 5.14 Percentage of population with employer-based and direct-purchase coverage: 0.58 Percentage of employed working in wholesale trade (ages 16 and over): 2.88 Percentage of households with other type of computer: 0.93 Percentage of male population now married and spouse absent (ages 15 and over): 6.21 Percentage of female population now married and spouse absent (ages 15 and over): 5.45 Percentage of employed working in other services, except public administration (ages 16 and over): 1.85 Percentage of civilian veterans that have some college education or an associate's degree (ages 25 and over): 27.14 Percentage of population who speak other Indo-European languages (ages 5 and over): 0.98 Percentage of population with other health insurance coverage combinations: 6.52 Percentage of population that moved within the same county in the past year (age 1 and over): 6.21 Percentage of employed working in finance and insurance, real estate, and rental and leasing (ages 16 and over): 1.13 75-84_e(x): 10.22 Percentage of civilian employed population working in public administration (ages 16 and over): 5.1 Percentage of population that moved from different county within same state in the past year (age 1 and over): 2.66 Percentage of employed working in transportation and warehousing, and in utilities (ages 16 and over): 4.37 Total grandchildren under 18 living with grandparent householder: 204.0 Percentage of employed working in construction (ages 16 and over): 13.23 Percentage of employed working in information services (ages 16 and over): 0.41 Percentage of households with broadband such as cable, fiber optic or DSL with no other type of Internet subscription: 2.93 Percentage of employed working in professional, scientific, management, administrative, and waste management services (ages 16 and over): 11.48 Percentage of employed working in arts, entertainment, recreation, and accommodation and food services (ages 16 and over): 22.03 Total population (between ages 16 and 19): 275.0 Percentage of population reporting multiple races: 4.46 Percentage of population with direct-purchase health insurance only: 4.09 Distance in miles to the nearest health clinic (FQHC, RHC), calculated using population weighted tract centroids: 1.31 Percentage of households with satellite internet service: 4.94 Distance in miles to the nearest emergency department, calculated using population weighted tract centroids: 2.41 Percentage of population who are private not-for-profit wage and salary workers (ages 16 and over): 13.9 Distance in miles to the nearest urgent care, calculated using population weighted tract centroids: 1.57 85 and older_e(x): 3.12 Distance in miles to the nearest designated trauma center, calculated using population weighted tract centroids: 2.41 Percentage of female population who worked in place of residence (ages 16 and over): 42.74 Percentage of population reporting Black or African American race alone: 25.72 Percentage of workers with at least 60-minute commute time (ages 16 and over): 14.03 Land area in square miles: 0.73 Percentage of employed working in manufacturing (ages 16 and over): 12.4 Percentage of occupied housing units with fuel oil heating: 0.0 Percentage of population consisting of U.S. citizens by naturalization: 5.39 OBESITY_Data_Value: 31.42 MAMMOUSE_Data_Value: 73.8 DEPRESSION_Data_Value: 23.24 CHOLSCREEN_Data_Value: 83.47999999999999 COLON_SCREEN_Data_Value: 69.67999999999999 Percentage of workers taking taxicab, motorcycle, bicycle, or other means to work (ages 16 and over): 1.41 Percentage of households with internet access without a subscription: 2.62 Percentage of children with a disability (ages 17 and below): 2.26 Percentage of population between ages 0-17: 26.03 Percentage of population between ages 30-44: 23.05 Percentage of population between ages 45-64: 23.46 Percentage of population ages 65 and over: 13.01 Percentage of population that is male: 56.18 Percentage of the civilian population consisting of veterans (ages 18 and over): 12.19 Per capita income (dollars, inflation-adjusted to data file year): 25042.0 Percentage of population with less than high school education (ages 25 and over): 14.78 Median gross rent (dollars): 1361.0 Percentage of households with only one occupant: 24.77 Percentage of population with no health insurance coverage: 15.53 .You are a smart model, state the 3 most probable problems this person may face based on the information provided and also give solutions to the respective problems?(List exactly 3 problems and their respective solutions)(Try to be concise and to the point)
"""

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=10000)[0], skip_special_tokens=True))